In [4]:
#Importing librabries
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from scipy.stats import randint, uniform

import warnings
warnings.filterwarnings("ignore")

In [5]:
#Initializing an empty dataframe for storing the data after each iteration
results_df = pd.DataFrame(columns=['n_estimators', 'learning_rate', 'max_depth', 'max_features', 'mean_test_score'])

In [7]:
#Loading the dataset
file_path = "./dataset.csv"
try:
    dataset = pd.read_csv(file_path)
except FileNotFoundError:
    print("File not found. Please check the path and try again.")
    exit(1)

In [8]:
#Verify if dataset is a dataframe
if not isinstance(dataset, pd.DataFrame):
    print("Dataset is not a dataframe. Please check the file and try again.")
    exit(1)
    